In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 49.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
import pandas as pd
from datasets import load_metric
import torch
# Load the model and tokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [4]:
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5_saved/t5_recipe_model").to("cuda")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/t5_saved/t5_recipe_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Load the dataset
test_df = pd.read_csv('NLP_Recipe_test.csv')

# Load BLEU metric
bleu_metric = load_metric('bleu')

<ipython-input-5-80f3897ea024>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [14]:
import pandas as pd
from datasets import load_metric
from transformers import AutoTokenizer, T5ForConditionalGeneration
import numpy as np
import torch


# Load the metrics
bleu_metric = load_metric('bleu')
meteor_metric = load_metric('meteor')
bertscore_metric = load_metric('bertscore', lang="en")
rouge_metric = load_metric('rouge')

# Load the model and tokenizer
model.to('cuda')  # Assuming you are using a GPU

# Lists to store scores
bleu_scores = []
meteor_scores = []
bertscore_f1_scores = []
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []

# Function to prepare data for T5 input
def prepare_data(row):
    ner_input = row['ner']
    prompt = f"generate full recipe steps and ingredients for: {ner_input}"
    return prompt

# Function to generate recipes and compute scores
def generate_and_score(index, row):
    prompt = prepare_data(row)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate outputs
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_beams=5, early_stopping=True)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Compute BLEU score
    true_combined = row['ingredients'] + " " + row['steps']
    reference = [true_combined.split()]
    candidate = generated_text.split()
    bleu_score = bleu_metric.compute(predictions=[candidate], references=[reference])['bleu']
    meteor_score = meteor_metric.compute(predictions=[generated_text], references=[true_combined])['meteor']
    bertscore_results = bertscore_metric.compute(predictions=[generated_text], references=[true_combined], lang='en')
    bertscore_f1 = np.mean(bertscore_results['f1'])
    rouge_score = rouge_metric.compute(predictions=[generated_text], references=[true_combined])

    # Append scores
    bleu_scores.append(bleu_score)
    meteor_scores.append(meteor_score)
    bertscore_f1_scores.append(bertscore_f1)
    rouge_1_scores.append(rouge_score['rouge1'].mid.fmeasure)
    rouge_2_scores.append(rouge_score['rouge2'].mid.fmeasure)
    rouge_l_scores.append(rouge_score['rougeL'].mid.fmeasure)

    return generated_text

# Iterate over the first 5 rows and compute scores
for index, row in test_df.head(5).iterrows():
    generate_and_score(index, row)

# Compute averages
avg_bleu = np.mean(bleu_scores)
avg_meteor = np.mean(meteor_scores)
avg_bertscore_f1 = np.mean(bertscore_f1_scores)
avg_rouge_1 = np.mean(rouge_1_scores)
avg_rouge_2 = np.mean(rouge_2_scores)
avg_rouge_l = np.mean(rouge_l_scores)

# Print average scores
print("Average BLEU Score:", avg_bleu)
print("Average METEOR Score:", avg_meteor)
print("Average BERTScore F1 Score:", avg_bertscore_f1)
print("Average ROUGE-1 Score:", avg_rouge_1)
print("Average ROUGE-2 Score:", avg_rouge_2)
print("Average ROUGE-L Score:", avg_rouge_l)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BLEU Score: 0.04822932434217149
Average METEOR Score: 0.20167020381436945
Average BERTScore F1 Score: 0.8479589819908142
Average ROUGE-1 Score: 0.320947226548108
Average ROUGE-2 Score: 0.15479197734395322
Average ROUGE-L Score: 0.2718234960801868
